# NYPD SHOOTING INCIDENTS DATASET:

a. To get the required columns, use this module:


1.   get_area_of_interest(df_spark, interested_columns)


b. Preprocessing pipeline: Pass your data through these functions. (if your columns fall in those categories)

1.   valid_date_check(date)
2.   valid_time_check(time)
3.   reverse_geo_code_boros(df_spark, Latitude, Longitude, Boro, lat_index, long_index)
4.   refine_age_group_race(df_spark, victim_age_group=None, suspect_age_group=None, suspect_race=None, victim_race=None)
5.   refine_sex_gender_impute(df_spark, suspect_age=None, suspect_gender=None, victim_age=None, victim_gender=None)
6.   refine_precinct_jur(df_spark, precinct=None, Jur_code=None)



In [ ]:
!pip install pyspark
!pip install openclean

     |████████████████████████████████| 281.3 MB 38 kB/s 
     |████████████████████████████████| 198 kB 64.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=d68a8d558d51cc1a6a2bcdcd10da8bccfcf57331bbf7afe7613fd80584aa2f50
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark
     |████████████████████████████████| 267 kB 7.0 MB/s 
     |████████████████████████████████| 109 kB 64.6 MB/s 
     |████████████████████████████████| 137 kB 52.7 MB/s 
     |████████████████████████████████| 69 kB 7.1 MB/s 
     |████████████████████████████████| 260 kB 38.9 MB/s 
     |████████████████████████████████| 206 kB 66.5 MB/s 
     |████████████████████████████████| 525 kB 47.1 MB/s 
     |████████████████████████████████| 596 kB 56.7 MB/s 
     |████████████████████████████████| 180 kB 49.9 MB/s 
     |████████████████████████████████| 149 kB 55.1 MB/s 
     |███████

In [ ]:
#importing packages required
from pyspark import SparkContext, SparkConf
import os
import requests
from six.moves import urllib
import sys
import pandas as pd
import matplotlib
import matplotlib as plt
import numpy as np
import scipy as sp
import IPython
from IPython import display
import sklearn
import random
import time
import warnings
import re
import matplotlib.pyplot as plt
%matplotlib inline
from openclean.pipeline import stream
from openclean.profiling.column import DefaultColumnProfiler
from openclean.data.source.socrata import Socrata
from openclean.pipeline import stream
from openclean.function.eval.datatype import IsDatetime
import datetime
import pandas as pd
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import udf, struct
from pyspark.sql.types import StringType

In [ ]:
from geopy.geocoders import ArcGIS
geocoder=ArcGIS()
#example:
geocoder.reverse('40.61157006600007, -73.74736517199995')

Location(11-64 Redfern Ave, Far Rockaway, New York 11691, USA, (40.61161616586613, -73.74738361194636, 0.0))

In [ ]:
#Creating Spark Session
sc = SparkContext.getOrCreate();
spark = SparkSession(sc)

In [ ]:
#MOUNT YOUR GOOGLE DRIVE FIRST

In [ ]:
#Downloading file from NYC Open Data

fn_src = 'https://data.cityofnewyork.us/api/views/833y-fsy8/rows.csv?accessType=DOWNLOAD'
fn_dst = '/content/NYPD_Shooting_Incident_Data__Historic_.csv'

from six.moves import urllib

if os.path.isfile(fn_dst):
    print('File has already been downloaded',fn_dst)
else:
    print('Fetching file. This may take a while..',fn_dst)
    urllib.request.urlretrieve(fn_src, fn_dst)
    print('File has been downloaded',fn_dst)

File %s has already been downloaded /content/NYPD_Shooting_Incident_Data__Historic_.csv


In [ ]:
#similarly, lets get them into pyspark rdd
def get_area_of_interest(df_spark, interested_columns):
  df_spark=df_spark.select(interested_columns)
  return df_spark

# 2. Module for date related columns

As the dataset is for the data from 2006 to 2025, we can see that there is data from unknown format of "1010-05-14" to the year 2025. We need to clean this. Over here, we remove the null values where the complaint date is <2006.

In [ ]:
# fileName='1010-05-14 00:00:00'
# # matches=re.search("([0-9]{4}\-[0-9]{2}\-[0-9]{2})", fileName)
# re.search(r'([0-9]{4}\-[0-9]{2}\-[0-9]{2})', fileName).group(0)

def valid_date_check(date):
  if date==None or date==" " or date=="":
      return False
  else:
    date_cpy=date
    date=date.split("/")
    try:
      month=int(date[0])
      day= int(date[1])
      year=int(date[2])
      if year>=2006 and year<=2025:
        try:
          refined_date=datetime.datetime(year, month, day)
          return True
        except:
          return False
      else:
        return False
    except:
      return False

# 3. Module for time related columns

Similarly, lets check for the time as well. Here we must have time between
the standard 24 hours.

In [ ]:
#Deleting invalid time
def valid_time_check(time):
  if time==None or time==" " or time=="":
    return False
  else :
    cpy_time=time
    time=time.split(":")
    try:
      hour=int(time[0])
      mins=int(time[1])
      secs= int(time[2])
      # if hours is 24 then change it to 0 hours
      if hour == 24 and mins== 0 and secs == 0:
        hour=0
      try:
        newTime= datetime.time(hour,mins,secs)
        return True
      except :
        return False
    except:
      return False

#4. Module for Age Group and Race columns
The module works for only those columns whose column names are passed

In [ ]:
def refine_age_group_race(df_spark, victim_age_group=None, suspect_age_group=None, suspect_race=None, victim_race=None):
  #params: dataframe, col names for the respective age, gender cols
  if victim_age_group:
    df_spark = df_spark.na.fill("UNKNOWN",subset=[victim_age_group])
  if suspect_age_group:
    df_spark = df_spark.na.fill("UNKNOWN",subset=[suspect_age_group])
  if suspect_race:
    df_spark = df_spark.na.fill("UNKNOWN",subset=[suspect_race])
  if victim_race:
    df_spark = df_spark.na.fill("UNKNOWN",subset=[victim_race])
  return df_spark

# 5. Module for Gender, Race Columns for suspects and victims

The module works for only those columns whose column names are passed

In [ ]:
def refine_sex_gender_impute(df_spark, suspect_age=None, suspect_gender=None, victim_age=None, victim_gender=None):
  #params: dataframe, col names for the respective age, gender cols
  if suspect_age:
    df_spark=df_spark.na.fill("U",subset=[suspect_age])
  if victim_age:
    df_spark=df_spark.na.fill("U",subset=[victim_age])
  if suspect_gender:
    df_spark = df_spark.na.fill("UNKNOWN",subset=[suspect_gender])
  if victim_gender:
    df_spark = df_spark.na.fill("UNKNOWN",subset=[victim_gender])
  return df_spark

# 6.a: Module for Precinct, Jurisdiction Code:
  dropping the null values

  The module works for only those columns whose column names are passed along with the df

In [ ]:
def valid_precinct_check(precinct):
  if precinct==None or precinct==" " or precinct=="":
    return False
  else :
    return True

def valid_jur_check(jur):
  if jur==None or jur==" " or jur=="":
    return False
  else :
    return True

# 6.b Module for Reverse Geocoding the boroughs using latitudes and longitudes.

1. First we will remove the rows where latitude, longitude and boroughs are null. (around 450 tuples removed)
2. Then, where the boroughs are empty, take the latitude and longitude value and reverse geocode it using the module "reverseGeocoder".
3. Impute the borough name retrived in the empty space.


### USING MASTER DATASET
In the case of geocoding, geocoder gives us the zipcodes based on the latitude and longitude values. Inturn, we can use the master dataset of zipcodes inorder to retrive the borough names



NOTE: The dataset can be downloaded from : https://data.beta.nyc/en/dataset/pediacities-nyc-neighborhoods/resource/7caac650-d082-4aea-9f9b-3681d568e8a5

In [ ]:
def reverse_geo_code_boros(df_spark, Latitude, Longitude, Boro, lat_index, long_index):
  #select data where we have to impute
  df_temp_boro_clean=df_spark.filter((df_spark[Latitude].isNotNull()) & (df_spark[Longitude].isNotNull()))
  boro_cleaner=df_temp_boro_clean.filter((df_temp_boro_clean[Boro].isNull())|(df_temp_boro_clean[Boro]=='NEW YORK'))

  # print("We have "+ str(boro_cleaner.count())+ " points to impute")
  print("___intializing Zip Code Look up ____")
  print("____ imputing the points ____")


  #use your path for master dataset here.
  df_zips=pd.read_csv(dst)
  zip_master={}
  zips=df_zips['zip']
  boro=df_zips['borough']
  for i, j in zip(zips, boro):
    zip_master[i]=j
  zip_master[10020]='Manhattan'
  zip_master[11249]='Brooklyn'

  def reverseGeoCoder(latitude, longitude):
    loc=geocoder.reverse(str(latitude)+', '+str(longitude), timeout=1000)
    zipCode=str(loc).split(",")[2][-5:]
    if not int(zipCode) in zip_master:
      boro="UNKNOWN"
    else:
      boro=zip_master[int(zipCode)]
    boro=boro.upper()
    return boro

  #creating UD function
  ud_func= udf(reverseGeoCoder, StringType())
  boro_cleaned_dataframe = boro_cleaner.withColumn(Boro, ud_func(boro_cleaner[lat_index], boro_cleaner[long_index]))

  #joining the imputed dataset to the maindataset and returning

  joiner_dataset=df_spark.filter((df_spark[Latitude].isNotNull()) & (df_spark[Boro]!='NEW YORK') & (df_spark[Longitude].isNotNull()) & (df_spark[Boro].isNotNull()))
  fin_df=joiner_dataset.union(boro_cleaned_dataframe)
  return fin_df

The size of dataset ~ 24k tuples. So, we need around 2000 data points for 95% confidence level with 2% interval. The size of data is almost 10% of the data. So we can get it into our df now

In [ ]:
df_spark=spark.read.option("header",True).csv(fn_dst,inferSchema=True)
df_spark=df_spark.sample(0.1)
df_spark.count()

2362

Lets profile first to see how many null values are present in which columns

In [ ]:
pandasDF = df_spark.toPandas()
ds=stream(pandasDF)

#Creating profile of our dataset
profiles = ds.profile(default_profiler=DefaultColumnProfiler)
profiles.stats()

,total,empty,distinct,uniqueness,entropy
INCIDENT_KEY,2362,0,2276,0.963590,11.130529
OCCUR_DATE,2362,0,1768,0.748518,10.645390
OCCUR_TIME,2362,0,911,0.385690,9.410089
BORO,2362,0,5,0.002117,1.953922
PRECINCT,2362,0,74,0.031329,5.552688
JURISDICTION_CODE,2362,0,3,0.001270,0.696481
LOCATION_DESC,2362,1363,26,0.026026,2.553088
STATISTICAL_MURDER_FLAG,2362,0,2,0.000847,0.713897
PERP_AGE_GROUP,2362,826,6,0.003906,2.024856
PERP_SEX,2362,822,3,0.001948,0.586243


## a. Select the columns that are common with the original dataset:
1. BORO
2. PRECINCT
3. JURISDICTION_CODE
4. PREP_AGE_GROUP
5. PERP_RACE
6. VIC_AGE_GROUP
7. VIC_SEX
8. Latitude
9. Longitude
10. VIC_RACE
11. PERP_SEX
12. OCCUR_DATE
13. OCCUR_TIME

We can consider the primary key along with this
14. INCIDENT_KEY

In [ ]:
interested_columns_1=['INCIDENT_KEY', 'OCCUR_TIME', 'OCCUR_DATE', 'BORO', 'PRECINCT', 'JURISDICTION_CODE', 'PERP_AGE_GROUP', 'PERP_RACE', 'PERP_SEX', 'VIC_AGE_GROUP', 'VIC_SEX', 'VIC_RACE', 'Latitude', 'Longitude']
df_spark=get_area_of_interest(df_spark, interested_columns_1)

## b. Lets pass the dataset through the preprocessing pipeline

In [ ]:
df_temp=df_spark.rdd

1. Date and Time

In [ ]:
df_temp_=df_temp.map(lambda x:(x, valid_date_check(x[2]))).filter(lambda x: x[1]==True)
df_temp=df_temp_.map(lambda x: x[0])

df_temp_=df_temp.map(lambda x:(x, valid_time_check(x[1]))).filter(lambda x: x[1]==True)
df_temp=df_temp_.map(lambda x: x[0])

2. Age group, Race, Gender imputation

In [ ]:
# #as this code requires the pyspark dataframe(Not the rdd)
df_temp=df_temp.toDF(schema=df_spark.schema)
df_temp=refine_age_group_race(df_temp, 'VIC_AGE_GROUP', 'PERP_AGE_GROUP', 'PERP_RACE', 'VIC_RACE')
df_temp=refine_sex_gender_impute(df_temp, None, "PERP_SEX", None, "VIC_SEX")

3. Geocoding

In [ ]:
df_spk=reverse_geo_code_boros(df_temp, 'Latitude', 'Longitude', 'BORO', -2, -1)

___intializing Zip Code Look up ____
____ imputing the points ____


4. Precinct and Jurisdiction Code

In [ ]:
df_temp=df_spk.rdd
df_temp_=df_temp.map(lambda x:(x, valid_precinct_check(x[4]))).filter(lambda x: x[1]==True)
df_temp=df_temp_.map(lambda x: x[0])

df_temp_=df_temp.map(lambda x:(x, valid_jur_check(x[5]))).filter(lambda x: x[1]==True)
df_temp=df_temp_.map(lambda x: x[0])

df_spark=df_temp.toDF(schema=df_spark.schema)

Lets profile the data now.

In [ ]:
pandasDF = df_spark.toPandas()
ds=stream(pandasDF)

#Creating profile of our dataset
profiles = ds.profile(default_profiler=DefaultColumnProfiler)
profiles.stats()

,total,empty,distinct,uniqueness,entropy
INCIDENT_KEY,2362,0,2276,0.963590,11.130529
OCCUR_TIME,2362,0,911,0.385690,9.410089
OCCUR_DATE,2362,0,1768,0.748518,10.645390
BORO,2362,0,5,0.002117,1.953922
PRECINCT,2362,0,74,0.031329,5.552688
JURISDICTION_CODE,2362,0,3,0.001270,0.696481
PERP_AGE_GROUP,2362,0,6,0.002540,1.835240
PERP_RACE,2362,0,6,0.002540,1.685145
PERP_SEX,2362,0,4,0.001693,1.314503
VIC_AGE_GROUP,2362,0,6,0.002540,1.721987
